In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"  # 가장 안정적인 GPU 1개만 지정


In [2]:
import tensorflow as tf
print("GPU 사용 가능:", tf.config.list_physical_devices('GPU'))
print("XLA 활성화 여부 (확인용):", tf.config.optimizer.get_experimental_options())
# 텐서플로우 버전
print("TensorFlow 버전:", tf.__version__)


2025-04-26 14:05:06.043860: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 14:05:06.058752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745643906.073357  407911 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745643906.077884  407911 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745643906.088667  407911 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

GPU 사용 가능: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
XLA 활성화 여부 (확인용): {'disable_model_pruning': False, 'disable_meta_optimizer': False}
TensorFlow 버전: 2.19.0


In [3]:
import tensorflow as tf  # 이 바로 다음에!

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU 메모리 점진 할당 설정 완료.")
    except RuntimeError as e:
        print("❌ 메모리 설정 실패:", e)


✅ GPU 메모리 점진 할당 설정 완료.


In [4]:
import tensorflow as tf

TRAIN_PATH = "/home/kangkr1002/facial_bone/Training"
VAL_PATH   = "/home/kangkr1002/facial_bone/Validation"
TEST_PATH  = "/home/kangkr1002/facial_bone/Test"
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='int',
    image_size=(380, 380),
    batch_size=32,
    shuffle=True,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_PATH,
    labels='inferred',
    label_mode='int',
    image_size=(380, 380),
    batch_size=32,
    shuffle=True,
    seed=42
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    labels='inferred',
    label_mode='int',
    image_size=(380, 380),
    batch_size=32,
    shuffle=True,
    seed=42
)

Found 16000 files belonging to 2 classes.


I0000 00:00:1745643915.603775  407911 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:d1:00.0, compute capability: 8.0


Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [5]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers, models


In [6]:
# EfficientNetB4 백본 (사전학습된 가중치 사용)
base_model = EfficientNetB4(
    input_shape=(380, 380, 3),
    include_top=False,  # classifier 부분 제거
    weights='imagenet'
)
base_model.trainable = False  # 처음엔 학습시키지 않음 (Transfer Learning)


In [7]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    # layers.Dropout(0.5),  # 잠깐 주석 처리해보기
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [8]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [9]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


I0000 00:00:1745643930.256847  408486 service.cc:152] XLA service 0x7fedf0003500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745643930.256881  408486 service.cc:160]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2025-04-26 14:05:31.076622: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745643934.138924  408486 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-26 14:05:39.215973: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23438', 12 bytes spill stores, 16 bytes spill loads

2025-04-26 14:05:39.245961: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23201'

  2/500 ━━━━━━━━━━━━━━━━━━━━ 28s 58ms/step - accuracy: 0.5000 - loss: 0.7214   

I0000 00:00:1745643954.298505  408486 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


499/500 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8271 - loss: 0.3654

2025-04-26 14:06:32.252599: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3966', 8 bytes spill stores, 8 bytes spill loads

2025-04-26 14:06:33.053520: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3966', 136 bytes spill stores, 136 bytes spill loads

2025-04-26 14:06:33.317844: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3966', 5016 bytes spill stores, 5024 bytes spill loads

2025-04-26 14:06:33.420129: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3966', 5360 bytes spill stores, 5412 bytes spill loads

2025-04-26 14:06:34.041104: E ex

500/500 ━━━━━━━━━━━━━━━━━━━━ 93s 117ms/step - accuracy: 0.8274 - loss: 0.3650 - val_accuracy: 0.9230 - val_loss: 0.1827
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9404 - loss: 0.1471 - val_accuracy: 0.9430 - val_loss: 0.1405
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9614 - loss: 0.1057 - val_accuracy: 0.9770 - val_loss: 0.0573
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9701 - loss: 0.0873 - val_accuracy: 0.9820 - val_loss: 0.0454
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9750 - loss: 0.0746 - val_accuracy: 0.9860 - val_loss: 0.0390
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.9758 - loss: 0.0662 - val_accuracy: 0.9850 - val_loss: 0.0449
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.9811 - loss: 0.0578 - val_accuracy: 0.9840 - val_loss: 0.0420
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9819 - loss: 0.0552 - val_accura

In [10]:
TEST_PATH  = "/home/kangkr1002/facial_bone/Test"

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    labels='inferred',
    label_mode='int',
    image_size=(380, 380),
    batch_size=32,
    shuffle=True,
    seed=42
)

# 모델 평가
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Found 1000 files belonging to 2 classes.
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9652 - loss: 0.0948
Test Loss: 0.0959
Test Accuracy: 0.9680


In [11]:
# 저장 모델 .pth
model.save('effcientnet_model.h5')